In [1]:
import pandas as pd

In [2]:
df_active = pd.DataFrame()
# find out the most active day in the whole file
for df_chunck in pd.read_csv('../../data/click_stream_new.csv', chunksize=100000):
    df_chunck['event_date'] = [ d[:10] for d in df_chunck['event_time'] ]
    df_chunck['count'] = 0
    df_filtered = df_chunck[['event_date', 'count']]
    df_filtered_groupped = df_filtered.groupby(['event_date']).count()
    
    df_most_active_day = df_filtered_groupped[df_filtered_groupped['count'] == df_filtered_groupped['count'].max()]
    df_most_active_day.reset_index(inplace=True)
    df_final = df_chunck[df_chunck['event_date'].eq(df_most_active_day.at[0, 'event_date'])]
    df_final.reset_index(inplace=True)
    
    if len(df_final) > len(df_active):
        df_active = df_final.copy()
        
df_active.head()

,index,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,event_date,count
0,12601518,bfda2978-2603-48d3-9849-2b7c2a36dfc7,HOMEPAGE,2022-07-31T23:21:37.923503Z,db7c30f5-7f5e-4596-8aee-23c31e6ed5a7,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31,0
1,12601519,bfda2978-2603-48d3-9849-2b7c2a36dfc7,ITEM_DETAIL,2022-07-31T23:30:18.923503Z,d24c7d65-8255-46df-b178-380069d48577,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31,0
2,12601721,388beacd-54b3-4b2e-a26d-cb7d99e1e694,HOMEPAGE,2022-07-31T20:42:50.103601Z,b11b4085-ce10-404f-9c5f-366ee6788c2f,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31,0
3,12601722,388beacd-54b3-4b2e-a26d-cb7d99e1e694,ADD_TO_CART,2022-07-31T20:48:19.103601Z,0c6968f8-e723-4d5a-9c07-fdf13d803bdd,MOBILE,58279.0,1.0,504859.0,NaN,NaN,NaN,NaN,2022-07-31,0
4,12601723,388beacd-54b3-4b2e-a26d-cb7d99e1e694,ADD_TO_CART,2022-07-31T20:54:18.103601Z,8ae0586a-b703-4e11-bc46-7a45af37ab7d,MOBILE,54508.0,1.0,433687.0,NaN,NaN,NaN,NaN,2022-07-31,0


In [3]:
df_active.shape

(1443, 15)

In [19]:
df_active_day = pd.DataFrame()
for df_chunck in pd.read_csv('../../data/click_stream_new.csv', chunksize=100000):
    df_chunck['event_date'] = [ d[:10] for d in df_chunck['event_time'] ]
    df_filtered = df_chunck[df_chunck['event_date'].eq(df_active.at[0, 'event_date'])]
    df_active_day = pd.concat([df_active_day, df_filtered], axis=0)

df_active_day.shape

(27759, 13)

In [23]:
df_active_day['event_time'] = pd.to_datetime(df_active_day['event_time'])

In [24]:
# offset from previous time
min_event_time = df_active_day['event_time'].min()
min_event_time

Timestamp('2022-07-31 00:00:08.361823+0000', tz='UTC')

In [25]:
df_active_day['event_time'] = pd.to_datetime(df_active_day['event_time'])
df_active_day.sort_values(by='event_time', inplace=True)
df_active_day.reset_index(inplace=True)
df_active_day.head()

,index,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,event_date
0,10164772,096fec88-b44c-4a27-ba14-48bcc1e670db,HOMEPAGE,2022-07-31 00:00:08.361823+00:00,881ee7c6-6d21-49c8-9e2f-e5cae9ea68e7,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31
1,10149939,12d8e1ff-a21b-4a6f-86f7-f5f31557986e,SCROLL,2022-07-31 00:00:19.191828+00:00,3057a1c1-828e-4d17-bf2c-a21292450613,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31
2,2321955,d7b4d7c4-17b3-48b7-92c3-73b484fd9f2f,SEARCH,2022-07-31 00:00:20.822075+00:00,649c483b-057b-4f5b-ac12-ac9e2ac216ff,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,2022-07-31
3,11963211,d5e152a1-e8d9-4d1a-a646-9ba7b31c5b96,SEARCH,2022-07-31 00:00:22.101671+00:00,9a060895-60e3-4ab3-8af0-566fb4f1dc9b,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,2022-07-31
4,10878721,04305057-f8b2-4265-bec1-8db695929418,SCROLL,2022-07-31 00:00:22.542968+00:00,7adf17f3-dece-4000-ae45-60c456504256,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31


In [26]:
df_active_day['event_offset'] = 0

for i, row in df_active_day.iterrows():
    if i > 0:
        df_active_day.at[i, 'event_offset'] = int((df_active_day.at[i, 'event_time'] - df_active_day.at[i-1, 'event_time']).total_seconds())

df_active_day.head()

,index,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,event_date,event_offset
0,10164772,096fec88-b44c-4a27-ba14-48bcc1e670db,HOMEPAGE,2022-07-31 00:00:08.361823+00:00,881ee7c6-6d21-49c8-9e2f-e5cae9ea68e7,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31,0
1,10149939,12d8e1ff-a21b-4a6f-86f7-f5f31557986e,SCROLL,2022-07-31 00:00:19.191828+00:00,3057a1c1-828e-4d17-bf2c-a21292450613,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31,10
2,2321955,d7b4d7c4-17b3-48b7-92c3-73b484fd9f2f,SEARCH,2022-07-31 00:00:20.822075+00:00,649c483b-057b-4f5b-ac12-ac9e2ac216ff,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,2022-07-31,1
3,11963211,d5e152a1-e8d9-4d1a-a646-9ba7b31c5b96,SEARCH,2022-07-31 00:00:22.101671+00:00,9a060895-60e3-4ab3-8af0-566fb4f1dc9b,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,2022-07-31,1
4,10878721,04305057-f8b2-4265-bec1-8db695929418,SCROLL,2022-07-31 00:00:22.542968+00:00,7adf17f3-dece-4000-ae45-60c456504256,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31,0


In [27]:
df_active.columns

Index(['index', 'session_id', 'event_name', 'event_time', 'event_id',
       'traffic_source', 'product_id', 'quantity', 'item_price',
       'payment_status', 'search_keywords', 'promo_code', 'promo_amount',
       'event_date', 'count', 'event_offset'],
      dtype='object')

In [28]:
df_active = df_active[['session_id', 'event_name', 'event_id', 'traffic_source', 'product_id', 'quantity', 'item_price',
       'payment_status', 'search_keywords', 'promo_code', 'promo_amount', 'event_offset']]
df_active.head()

,session_id,event_name,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,event_offset
0,bfda2978-2603-48d3-9849-2b7c2a36dfc7,HOMEPAGE,db7c30f5-7f5e-4596-8aee-23c31e6ed5a7,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,bfda2978-2603-48d3-9849-2b7c2a36dfc7,ITEM_DETAIL,d24c7d65-8255-46df-b178-380069d48577,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
2,388beacd-54b3-4b2e-a26d-cb7d99e1e694,HOMEPAGE,b11b4085-ce10-404f-9c5f-366ee6788c2f,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,388beacd-54b3-4b2e-a26d-cb7d99e1e694,ADD_TO_CART,0c6968f8-e723-4d5a-9c07-fdf13d803bdd,MOBILE,58279.0,1.0,504859.0,NaN,NaN,NaN,NaN,1.0
4,388beacd-54b3-4b2e-a26d-cb7d99e1e694,ADD_TO_CART,8ae0586a-b703-4e11-bc46-7a45af37ab7d,MOBILE,54508.0,1.0,433687.0,NaN,NaN,NaN,NaN,0.0


In [45]:
df_active.to_csv('../data/one-day-stream-filtered.csv', index=False)

In [31]:
df_transac = pd.read_csv('../../data/transaction_new.csv')
df_transac.head()

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price
0,2018-07-29T15:22:01.458193Z,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832,54728,1,191247
1,2018-07-30T12:40:22.365620Z,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526,16193,1,145526
2,2018-09-15T11:51:17.365620Z,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696,53686,4,135174
3,2018-11-01T11:23:48.365620Z,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012,20228,1,271012
4,2018-12-18T11:20:30.365620Z,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753,55220,1,198753


In [32]:
df_transac['event_date'] = [ d[:10] for d in df_transac['created_at'] ]
df_transac_filtered = df_transac[df_transac['event_date'].eq(df_active_day.at[0, 'event_date'])]
df_transac_filtered.shape

(1667, 17)

In [33]:
df_transac_filtered.head()

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price,event_date
13024,2022-07-31T03:24:58.759811Z,4943,22ffc671-3862-4c58-9367-0eb1948828d8,6a5e2195-5493-4cae-9870-a7543cfc8cf0,Credit Card,Success,4657,WEEKENDSERU,0,2022-08-02T10:00:12.864118Z,-6.172311,106.864336,1233609,16426,5,196316,2022-07-31
13025,2022-07-31T03:24:58.759811Z,4943,22ffc671-3862-4c58-9367-0eb1948828d8,6a5e2195-5493-4cae-9870-a7543cfc8cf0,Credit Card,Success,4657,WEEKENDSERU,0,2022-08-02T10:00:12.864118Z,-6.172311,106.864336,1233609,48806,1,256686,2022-07-31
13862,2022-07-31T01:49:16.627629Z,91500,c63b10dc-e7ba-4e25-b9c7-72a7643c4647,57013871-ed78-4e02-9452-2be7743c643c,Credit Card,Failed,3834,LIBURDONG,0,2022-08-03T16:09:15.564095Z,-6.108863,106.879296,129999,21277,1,133833,2022-07-31
14205,2022-07-31T08:21:55.276096Z,12613,0f454d64-3750-4446-aee6-5794f22f9b5f,c01ccb7c-fe1e-4392-9223-c7bc19f82fbb,Debit Card,Success,5885,WEEKENDSERU,10000,2022-08-02T04:07:32.491051Z,1.059123,103.836039,375403,12891,1,371288,2022-07-31
14369,2022-07-31T23:11:06.004213Z,76270,01c98a95-1fea-4dfc-95b3-7abffc682c92,c2a957a6-1fd0-4ba0-8edb-a0d65981358d,OVO,Success,0,NaN,10000,2022-08-06T06:08:53.336690Z,-7.570575,110.255343,110988,48879,1,100988,2022-07-31


In [34]:
df_transac_filtered.sort_values(by='created_at', inplace=True) 
df_transac_filtered.reset_index(inplace=True)

/tmp/ipykernel_966472/2848889946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transac_filtered.sort_values(by='created_at', inplace=True)


In [35]:
df_transac_filtered['created_at'] = pd.to_datetime(df_transac_filtered['created_at'])

/tmp/ipykernel_966472/1372924631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transac_filtered['created_at'] = pd.to_datetime(df_transac_filtered['created_at'])


In [36]:
df_transac_filtered['event_offset'] = 0

for index, row in df_transac_filtered.iterrows():
    if index > 0:
        df_transac_filtered.at[index, 'event_offset'] = int((df_transac_filtered.at[index, 'created_at'] - df_transac_filtered.at[index-1, 'created_at']).total_seconds())

df_transac_filtered.head(5)

/tmp/ipykernel_966472/627857978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transac_filtered['event_offset'] = 0


,index,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price,event_date,event_offset
0,271002,2022-07-31 00:01:57.082014+00:00,56249,8b814fb1-866f-490e-8a86-336a0ec07ce6,764e800f-0e4d-41f4-86b7-ac125e09f8f7,OVO,Success,0,NaN,10000,2022-08-02T01:02:41.871378Z,-6.760483,107.145038,234659,59595,1,224659,2022-07-31,0
1,943970,2022-07-31 00:02:30.514880+00:00,88526,afd64790-3605-4d36-b5b0-abd2e3a5823d,8bdc6d05-b7c2-422c-a74c-b636ae0bc660,Credit Card,Success,3881,WEEKENDSERU,10000,2022-08-02T20:33:03.125175Z,0.458978,117.285013,283036,17183,1,276917,2022-07-31,33
2,412052,2022-07-31 00:04:20.909479+00:00,41121,798a298b-2966-4d69-8b9a-fadc4f87b857,eacfe57e-6c19-413a-a3eb-4c79f94a1f33,Credit Card,Success,0,NaN,0,2022-08-04T07:20:51.990247Z,-6.186397,106.872607,92318,45127,1,92318,2022-07-31,110
3,865745,2022-07-31 00:06:17.377315+00:00,14151,21656164-fa7c-43c7-a369-0fb05a980299,83a01b81-fe42-43bb-97d1-93244c847594,Gopay,Success,0,NaN,0,2022-08-02T17:44:45.837917Z,-0.275616,110.529582,527040,44184,1,247548,2022-07-31,116
4,865744,2022-07-31 00:06:17.377315+00:00,14151,21656164-fa7c-43c7-a369-0fb05a980299,83a01b81-fe42-43bb-97d1-93244c847594,Gopay,Success,0,NaN,0,2022-08-02T17:44:45.837917Z,-0.275616,110.529582,527040,12913,1,279492,2022-07-31,0


In [37]:
df_transac_filtered.columns

Index(['index', 'created_at', 'customer_id', 'booking_id', 'session_id',
       'payment_method', 'payment_status', 'promo_amount', 'promo_code',
       'shipment_fee', 'shipment_date_limit', 'shipment_location_lat',
       'shipment_location_long', 'total_amount', 'product_id', 'quantity',
       'item_price', 'event_date', 'event_offset'],
      dtype='object')

In [38]:
df_transac_filtered = df_transac_filtered[['customer_id', 'booking_id', 'session_id',
       'payment_method', 'payment_status', 'promo_amount', 'promo_code',
       'shipment_fee', 'shipment_location_lat', 'shipment_location_long', 'total_amount', 'product_id', 'quantity',
       'item_price', 'event_offset']]
df_transac_filtered.head()

,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price,event_offset
0,56249,8b814fb1-866f-490e-8a86-336a0ec07ce6,764e800f-0e4d-41f4-86b7-ac125e09f8f7,OVO,Success,0,NaN,10000,-6.760483,107.145038,234659,59595,1,224659,0
1,88526,afd64790-3605-4d36-b5b0-abd2e3a5823d,8bdc6d05-b7c2-422c-a74c-b636ae0bc660,Credit Card,Success,3881,WEEKENDSERU,10000,0.458978,117.285013,283036,17183,1,276917,33
2,41121,798a298b-2966-4d69-8b9a-fadc4f87b857,eacfe57e-6c19-413a-a3eb-4c79f94a1f33,Credit Card,Success,0,NaN,0,-6.186397,106.872607,92318,45127,1,92318,110
3,14151,21656164-fa7c-43c7-a369-0fb05a980299,83a01b81-fe42-43bb-97d1-93244c847594,Gopay,Success,0,NaN,0,-0.275616,110.529582,527040,44184,1,247548,116
4,14151,21656164-fa7c-43c7-a369-0fb05a980299,83a01b81-fe42-43bb-97d1-93244c847594,Gopay,Success,0,NaN,0,-0.275616,110.529582,527040,12913,1,279492,0


In [46]:
df_transac_filtered.to_csv('../data/one-day-transactions-filtered.csv', index=False)